In [1]:
import pandas as pd
features = pd.read_csv('../0 - ModelData/50MostImp+CombinedFeatures.csv')
AllFeatures = pd.read_csv('../../../Fucking Final Data/Working_DataFrame.csv')
target = pd.read_csv('../../../Fucking Final Data/CleanedFinal_Target_Data_Combined_resid_Trend')

In [2]:
# Check for NaN values in features dataframe
features_nan_counts = features.isnull().sum()
print("NaN values in features dataframe:")
print(features_nan_counts[features_nan_counts > 0])

# Check for NaN values in AllFeatures dataframe
AllFeatures_nan_counts = AllFeatures.isnull().sum()
print("\nNaN values in AllFeatures dataframe:")
print(AllFeatures_nan_counts[AllFeatures_nan_counts > 0])

# Check for NaN values in target dataframe
target_nan_counts = target.isnull().sum()
print("\nNaN values in target dataframe:")
print(target_nan_counts[target_nan_counts > 0])

NaN values in features dataframe:
Series([], dtype: int64)

NaN values in AllFeatures dataframe:
Series([], dtype: int64)

NaN values in target dataframe:
Series([], dtype: int64)


In [3]:
# Function to replace outliers using IQR with moving average
def replace_outliers_with_moving_avg(df, column, window=3):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Calculate the moving average
    df['moving_avg'] = df[column].rolling(window=window, center=True).mean()
    
    # Replace outliers with the moving average
    df[column] = df.apply(lambda row: row['moving_avg'] if row[column] < lower_bound or row[column] > upper_bound else row[column], axis=1)
    
    # Drop the temporary moving average column
    df.drop(columns=['moving_avg'], inplace=True)
    
    return df

# Replace outliers in the target dataset
target = replace_outliers_with_moving_avg(target, 'PM10_Combined_Trend_Residual')

# Ensure the features and target datasets are aligned
features = features[features['Datum'].isin(target['Datum'])]
target = target[target['Datum'].isin(features['Datum'])]
AllFeatures = AllFeatures[AllFeatures['Datum'].isin(target['Datum'])]

print(target.head())


                       Datum  PM10_Combined_Trend_Residual
0  2022-01-01 00:00:00+00:00                           NaN
1  2022-01-01 01:00:00+00:00                     53.126838
2  2022-01-01 02:00:00+00:00                     36.328107
3  2022-01-01 03:00:00+00:00                     24.801767
4  2022-01-01 04:00:00+00:00                      9.683660


In [4]:
from sklearn.preprocessing import StandardScaler
from traitlets import All

# Create lagged target variables
features['PM10_1h_lag'] = target['PM10_Combined_Trend_Residual'].shift(1)
features['PM10_2h_lag'] = target['PM10_Combined_Trend_Residual'].shift(2)
features['PM10_24h_lag'] = target['PM10_Combined_Trend_Residual'].shift(24)
AllFeatures['PM10_1h_lag'] = target['PM10_Combined_Trend_Residual'].shift(1)
AllFeatures['PM10_2h_lag'] = target['PM10_Combined_Trend_Residual'].shift(2)
AllFeatures['PM10_24h_lag'] = target['PM10_Combined_Trend_Residual'].shift(24)

# Drop rows with NaN values
features.dropna(inplace=True)
AllFeatures.dropna(inplace=True)
target = target[target['Datum'].isin(features['Datum'])]
target = target[target['Datum'].isin(AllFeatures['Datum'])]

features.drop(columns=['Datum'], inplace=True)
AllFeatures.drop(columns=['Datum'], inplace=True)

# Standardize the features dataset
scaler = StandardScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
AllFeatures_scaled = pd.DataFrame(scaler.fit_transform(AllFeatures), columns=AllFeatures.columns)

# Ensure there are no NaN values in the target dataset
target = target.dropna()

print(features_scaled.columns)
print(AllFeatures_scaled.columns)

Index(['Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]', 'Stromverbrauch', '350n_sumLief',
       '350v_sumLW', '352v_sumPW', '352v_sumLief', '354n_sumLW', '402v_sumPW',
       '402n_sumLief', '402n_sumLW', '403n_sumLW', '403v_sumPW', '403v_sumLW',
       '405v_sumLief', '405v_sumLW', '406n_sumLW', '406v_sumLief',
       '406v_sumLW', '408n_sumPW', '408n_sumLief', '408n_sumLW', '409v_sumLW',
       '415v_sumLief', '415v_sumLW', '415n_sumLW', '417n_sumPW',
       '417n_sumLief', '417n_sumLW', '419v_sumLW', '419n_sumPW',
       '419n_sumLief', '420n_sumLief', '420n_sumLW', '420v_sumLW',
       '653n_sumLW', '659v_sumLW', '659n_sumLief', '659n_sumLW', '660v_sumPW',
       '660v_sumLW', '660n_sumPW', '660n_sumLW', '84111104n_sumLief',
       '84111104v_sumLief', '84111108v_sumLief', 'Gasverbrauch', 'Traffic',
       'Basel Temperature [2 m elevation corrected]_x_Basel Temperature [2 m elevatio

In [5]:
features_scaled.to_csv('../0 - ModelData/FINAL:50MostImp+CombinationFeatures+Lagged.csv', index=False)
AllFeatures_scaled.to_csv('../0 - ModelData/FINAL:AllFeatures+Lagged.csv', index=False)
target.to_csv('../0 - ModelData/FINAL:Target---OutliersTreated.csv', index=False)